# Subject Specific Classification 
Because the previous results failed to perform with regards to the null model during the last submission deadline, let's try to use subject specific data

In [1]:
import tensorflow as tf
# tf.enable_eager_execution()
# Helperfunctions to make your feature definition more readable

In [2]:
import tf_mapper
from importlib import reload
reload(tf_mapper)
from tf_mapper import get_batched_dataset

In [3]:
train_indices, valid_indices, test_indices = tf_mapper.get_indices()

In [4]:
len(train_indices)

1044

In [5]:
all_subjects = tf_mapper.get_subjects()

In [6]:
def get_model():
    num_cnn_layers = 5
    num_lstm_layers = 1
    num_lin_layers = 2
    inputLayer = tf.keras.layers.Input((1500, 3))
    x = inputLayer


    for i in range(num_cnn_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv1D(16, (3,), padding="same")(x)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.MaxPool1D((2,))(x)
    for j in range(num_lstm_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.CuDNNLSTM(256, return_sequences=True)(x)
        x = tf.keras.layers.LeakyReLU()(x)


    x = tf.keras.layers.Flatten(name="flatten_encoder_lstm")(x)
    x = tf.keras.layers.Dense(200)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    x_shared_flattened = x

    #one_off
    x = x_shared_flattened 
    for k in range(num_lin_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(256)(x)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1)(x)
    x_on_off = tf.keras.layers.ReLU(name="on_off", max_value=4)(x)

    #tremor
    x = x_shared_flattened 
    for k in range(num_lin_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(100)(x)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1)(x)
    x_dyskinesia = tf.keras.layers.ReLU(name="dyskinesia", max_value=4)(x)

    #montage classify
    x = x_shared_flattened 
    for k in range(num_lin_layers):
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(100)(x)
        x = tf.keras.layers.LeakyReLU()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1)(x)
    x_tremor = tf.keras.layers.ReLU(name="tremor", max_value=4)(x)



    model = tf.keras.Model(inputs=inputLayer, outputs=[x_on_off, x_dyskinesia, x_tremor])
    model.compile("adam", loss=["mean_squared_error", "mean_squared_error","mean_squared_error", ])
    return model

In [7]:
all_subjects = tf_mapper.get_subjects()
all_subjects

[1004,
 1006,
 1007,
 1019,
 1020,
 1023,
 1032,
 1034,
 1038,
 1039,
 1043,
 1044,
 1046,
 1048,
 1049,
 1051]

In [8]:
import pickle as pkl

In [11]:
len(all_subjects)

16

In [23]:
reload(tf_mapper)
for subject in all_subjects:
    model = get_model()
    train_indices, valid_indices, test_indices = tf_mapper.get_splits_per_subject(subject)
    train_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=list(train_indices), batch_size=128, map_example=tf_mapper.map_example_to_simple_train, max_queue_size=20, cache=True)
    valid_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=list(valid_indices), batch_size=512, map_example=tf_mapper.map_example_to_simple_no_augment, max_queue_size=20, cache=True)
    test_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=list(test_indices), batch_size=512, map_example=tf_mapper.map_example_to_simple_no_augment, max_queue_size=20, cache=True)
    modelCheckpoint = tf.keras.callbacks.ModelCheckpoint(f"/n/scratch2/ms994/cnnlstm3{subject}.h5", save_best_only=True, verbose=True)
    reduceLR = tf.keras.callbacks.ReduceLROnPlateau(patience=4, factor=0.2, verbose=True)
    earlyStopping = tf.keras.callbacks.EarlyStopping(patience=20, min_delta=0.01)
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda i, lr: lr*0.8)
    history = model.fit(train_data, steps_per_epoch=250, epochs=200, validation_data=valid_data, validation_steps=100, callbacks=[modelCheckpoint, reduceLR, earlyStopping])
    pkl.dump(history.history, open(f"/n/scratch2/ms994/cnnlstm3{subject}_history.pkl", "wb"))

Epoch 1/200
248/250 [============================>.] - ETA: 0s - loss: 2.4861 - on_off_loss: 1.1143 - dyskinesia_loss: 0.6451 - tremor_loss: 0.7267
Epoch 00001: val_loss improved from inf to 3.80798, saving model to /n/scratch2/ms994/cnnlstm31004.h5
250/250 [==============================] - 138s 552ms/step - loss: 2.4805 - on_off_loss: 1.1109 - dyskinesia_loss: 0.6451 - tremor_loss: 0.7245 - val_loss: 3.8080 - val_on_off_loss: 1.4143 - val_dyskinesia_loss: 1.1713 - val_tremor_loss: 1.2224
Epoch 2/200
249/250 [============================>.] - ETA: 0s - loss: 1.7158 - on_off_loss: 0.7844 - dyskinesia_loss: 0.4246 - tremor_loss: 0.5069
Epoch 00002: val_loss did not improve from 3.80798
250/250 [==============================] - 12s 49ms/step - loss: 1.7171 - on_off_loss: 0.7842 - dyskinesia_loss: 0.4254 - tremor_loss: 0.5074 - val_loss: 3.8999 - val_on_off_loss: 1.5482 - val_dyskinesia_loss: 1.2137 - val_tremor_loss: 1.1379
Epoch 3/200
249/250 [============================>.] - ETA: 0s 

Epoch 19/200
249/250 [============================>.] - ETA: 0s - loss: 0.1753 - on_off_loss: 0.0710 - dyskinesia_loss: 0.0372 - tremor_loss: 0.0671
Epoch 00019: val_loss did not improve from 3.80798
250/250 [==============================] - 12s 48ms/step - loss: 0.1755 - on_off_loss: 0.0712 - dyskinesia_loss: 0.0372 - tremor_loss: 0.0671 - val_loss: 4.1818 - val_on_off_loss: 1.5399 - val_dyskinesia_loss: 1.2251 - val_tremor_loss: 1.4168
Epoch 20/200
249/250 [============================>.] - ETA: 0s - loss: 0.1765 - on_off_loss: 0.0714 - dyskinesia_loss: 0.0373 - tremor_loss: 0.0678
Epoch 00020: val_loss did not improve from 3.80798
250/250 [==============================] - 12s 48ms/step - loss: 0.1764 - on_off_loss: 0.0714 - dyskinesia_loss: 0.0372 - tremor_loss: 0.0678 - val_loss: 4.1761 - val_on_off_loss: 1.5327 - val_dyskinesia_loss: 1.2250 - val_tremor_loss: 1.4183
Epoch 21/200
249/250 [============================>.] - ETA: 0s - loss: 0.1826 - on_off_loss: 0.0738 - dyskinesia_

Epoch 16/200
249/250 [============================>.] - ETA: 0s - loss: 0.1085 - on_off_loss: 0.0358 - dyskinesia_loss: 0.0207 - tremor_loss: 0.0519
Epoch 00016: val_loss did not improve from 6.31532
250/250 [==============================] - 12s 49ms/step - loss: 0.1084 - on_off_loss: 0.0358 - dyskinesia_loss: 0.0207 - tremor_loss: 0.0519 - val_loss: 7.9998 - val_on_off_loss: 4.5698 - val_dyskinesia_loss: 0.7118 - val_tremor_loss: 2.7182
Epoch 17/200
249/250 [============================>.] - ETA: 0s - loss: 0.1091 - on_off_loss: 0.0364 - dyskinesia_loss: 0.0207 - tremor_loss: 0.0521
Epoch 00017: val_loss did not improve from 6.31532

Epoch 00017: ReduceLROnPlateau reducing learning rate to 1.6000001778593287e-06.
250/250 [==============================] - 12s 49ms/step - loss: 0.1093 - on_off_loss: 0.0364 - dyskinesia_loss: 0.0207 - tremor_loss: 0.0522 - val_loss: 8.0444 - val_on_off_loss: 4.6021 - val_dyskinesia_loss: 0.7157 - val_tremor_loss: 2.7266
Epoch 18/200
249/250 [==========

Epoch 13/200
248/250 [============================>.] - ETA: 0s - loss: 1.5271 - on_off_loss: 0.5924 - dyskinesia_loss: 0.4467 - tremor_loss: 0.4881
Epoch 00013: val_loss did not improve from 3.41878

Epoch 00013: ReduceLROnPlateau reducing learning rate to 8.000000525498762e-06.
250/250 [==============================] - 12s 49ms/step - loss: 1.5260 - on_off_loss: 0.5924 - dyskinesia_loss: 0.4463 - tremor_loss: 0.4873 - val_loss: 3.8286 - val_on_off_loss: 2.5457 - val_dyskinesia_loss: 0.5673 - val_tremor_loss: 0.7156
Epoch 14/200
248/250 [============================>.] - ETA: 0s - loss: 1.4819 - on_off_loss: 0.5718 - dyskinesia_loss: 0.4348 - tremor_loss: 0.4753
Epoch 00014: val_loss did not improve from 3.41878
250/250 [==============================] - 12s 49ms/step - loss: 1.4805 - on_off_loss: 0.5710 - dyskinesia_loss: 0.4341 - tremor_loss: 0.4754 - val_loss: 3.8556 - val_on_off_loss: 2.5885 - val_dyskinesia_loss: 0.5693 - val_tremor_loss: 0.6978
Epoch 15/200
249/250 [===========

Epoch 10/200
248/250 [============================>.] - ETA: 0s - loss: 0.2008 - on_off_loss: 0.0573 - dyskinesia_loss: 0.0689 - tremor_loss: 0.0746
Epoch 00010: val_loss did not improve from 2.49572
250/250 [==============================] - 12s 49ms/step - loss: 0.2007 - on_off_loss: 0.0571 - dyskinesia_loss: 0.0690 - tremor_loss: 0.0746 - val_loss: 4.3084 - val_on_off_loss: 0.4835 - val_dyskinesia_loss: 0.6564 - val_tremor_loss: 3.1685
Epoch 11/200
249/250 [============================>.] - ETA: 0s - loss: 0.1971 - on_off_loss: 0.0548 - dyskinesia_loss: 0.0682 - tremor_loss: 0.0740
Epoch 00011: val_loss did not improve from 2.49572
250/250 [==============================] - 12s 50ms/step - loss: 0.1970 - on_off_loss: 0.0547 - dyskinesia_loss: 0.0681 - tremor_loss: 0.0741 - val_loss: 4.4006 - val_on_off_loss: 0.5242 - val_dyskinesia_loss: 0.7329 - val_tremor_loss: 3.1434
Epoch 12/200
249/250 [============================>.] - ETA: 0s - loss: 0.1879 - on_off_loss: 0.0541 - dyskinesia_

...

Epoch 3/200
248/250 [============================>.] - ETA: 0s - loss: 0.6852 - on_off_loss: 0.3475 - dyskinesia_loss: 0.1031 - tremor_loss: 0.2345
Epoch 00003: val_loss improved from 5.07827 to 4.81860, saving model to /n/scratch2/ms994/cnnlstm31046.h5
250/250 [==============================] - 14s 58ms/step - loss: 0.6840 - on_off_loss: 0.3469 - dyskinesia_loss: 0.1028 - tremor_loss: 0.2343 - val_loss: 4.8186 - val_on_off_loss: 1.3967 - val_dyskinesia_loss: 1.7414 - val_tremor_loss: 1.6805
Epoch 4/200
248/250 [============================>.] - ETA: 0s - loss: 0.5550 - on_off_loss: 0.2695 - dyskinesia_loss: 0.0830 - tremor_loss: 0.2025
Epoch 00004: val_loss did not improve from 4.81860
250/250 [==============================] - 14s 56ms/step - loss: 0.5540 - on_off_loss: 0.2690 - dyskinesia_loss: 0.0829 - tremor_loss: 0.2021 - val_loss: 4.8562 - val_on_off_loss: 1.5873 - val_dyskinesia_loss: 1.7869 - val_tremor_loss: 1.4820
Epoch 5/200
248/250 [============================>.] - ETA: 0

Epoch 21/200
248/250 [============================>.] - ETA: 0s - loss: 0.0763 - on_off_loss: 0.0341 - dyskinesia_loss: 0.0113 - tremor_loss: 0.0308
Epoch 00021: val_loss did not improve from 4.81860
250/250 [==============================] - 14s 56ms/step - loss: 0.0762 - on_off_loss: 0.0341 - dyskinesia_loss: 0.0114 - tremor_loss: 0.0307 - val_loss: 5.5227 - val_on_off_loss: 1.7081 - val_dyskinesia_loss: 1.9607 - val_tremor_loss: 1.8539
Epoch 22/200
248/250 [============================>.] - ETA: 0s - loss: 0.0771 - on_off_loss: 0.0348 - dyskinesia_loss: 0.0112 - tremor_loss: 0.0311
Epoch 00022: val_loss did not improve from 4.81860
250/250 [==============================] - 14s 55ms/step - loss: 0.0772 - on_off_loss: 0.0348 - dyskinesia_loss: 0.0112 - tremor_loss: 0.0312 - val_loss: 5.5180 - val_on_off_loss: 1.7029 - val_dyskinesia_loss: 1.9566 - val_tremor_loss: 1.8585
Epoch 23/200
248/250 [============================>.] - ETA: 0s - loss: 0.0741 - on_off_loss: 0.0335 - dyskinesia_

Epoch 16/200
248/250 [============================>.] - ETA: 0s - loss: 0.2619 - on_off_loss: 0.0946 - dyskinesia_loss: 0.0716 - tremor_loss: 0.0958
Epoch 00016: val_loss did not improve from 5.03395
250/250 [==============================] - 14s 56ms/step - loss: 0.2619 - on_off_loss: 0.0945 - dyskinesia_loss: 0.0718 - tremor_loss: 0.0956 - val_loss: 6.3658 - val_on_off_loss: 3.9179 - val_dyskinesia_loss: 1.3665 - val_tremor_loss: 1.0815
Epoch 17/200
249/250 [============================>.] - ETA: 0s - loss: 0.2487 - on_off_loss: 0.0889 - dyskinesia_loss: 0.0678 - tremor_loss: 0.0919
Epoch 00017: val_loss did not improve from 5.03395
250/250 [==============================] - 14s 56ms/step - loss: 0.2486 - on_off_loss: 0.0888 - dyskinesia_loss: 0.0678 - tremor_loss: 0.0920 - val_loss: 6.3304 - val_on_off_loss: 3.8723 - val_dyskinesia_loss: 1.3722 - val_tremor_loss: 1.0860
Epoch 18/200
248/250 [============================>.] - ETA: 0s - loss: 0.2531 - on_off_loss: 0.0901 - dyskinesia_

Epoch 12/200
249/250 [============================>.] - ETA: 0s - loss: 0.1056 - on_off_loss: 0.0336 - dyskinesia_loss: 0.0222 - tremor_loss: 0.0497
Epoch 00012: val_loss did not improve from 5.83774
250/250 [==============================] - 15s 59ms/step - loss: 0.1056 - on_off_loss: 0.0337 - dyskinesia_loss: 0.0222 - tremor_loss: 0.0497 - val_loss: 5.9643 - val_on_off_loss: 4.5002 - val_dyskinesia_loss: 0.9339 - val_tremor_loss: 0.5302
Epoch 13/200
249/250 [============================>.] - ETA: 0s - loss: 0.1040 - on_off_loss: 0.0333 - dyskinesia_loss: 0.0220 - tremor_loss: 0.0487
Epoch 00013: val_loss did not improve from 5.83774
250/250 [==============================] - 14s 57ms/step - loss: 0.1039 - on_off_loss: 0.0333 - dyskinesia_loss: 0.0220 - tremor_loss: 0.0487 - val_loss: 5.9553 - val_on_off_loss: 4.4716 - val_dyskinesia_loss: 0.9436 - val_tremor_loss: 0.5401
Epoch 14/200
248/250 [============================>.] - ETA: 0s - loss: 0.1001 - on_off_loss: 0.0312 - dyskinesia_

Epoch 7/200
248/250 [============================>.] - ETA: 0s - loss: 2.4972 - on_off_loss: 1.2219 - dyskinesia_loss: 0.6368 - tremor_loss: 0.6385
Epoch 00007: val_loss did not improve from 4.31574

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
250/250 [==============================] - 48s 192ms/step - loss: 2.4946 - on_off_loss: 1.2208 - dyskinesia_loss: 0.6364 - tremor_loss: 0.6374 - val_loss: 5.5828 - val_on_off_loss: 3.1318 - val_dyskinesia_loss: 1.2851 - val_tremor_loss: 1.1659
Epoch 8/200
249/250 [============================>.] - ETA: 0s - loss: 1.7883 - on_off_loss: 0.8356 - dyskinesia_loss: 0.4652 - tremor_loss: 0.4876
Epoch 00008: val_loss did not improve from 4.31574
250/250 [==============================] - 18s 73ms/step - loss: 1.7877 - on_off_loss: 0.8353 - dyskinesia_loss: 0.4648 - tremor_loss: 0.4876 - val_loss: 5.8821 - val_on_off_loss: 3.3881 - val_dyskinesia_loss: 1.3445 - val_tremor_loss: 1.1494
Epoch 9/200
249/250 [============

In [10]:
results = []
for subject in all_subjects:
    model = tf.keras.models.load_model(f"/n/scratch2/ms994/cnnlstm3{subject}.h5")
    train_indices, valid_indices, test_indices = tf_mapper.get_splits_per_subject(subject)
    test_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=list(test_indices), batch_size=1, map_example=tf_mapper.map_example_to_simple_no_augment, max_queue_size=20, cache=True)
    results.append(model.evaluate(test_data, steps=len(test_indices)*40))

1840/1840 [==============================] - 34s 19ms/step - loss: 3.1754 - on_off_loss: 1.9317 - dyskinesia_loss: 0.5234 - tremor_loss: 0.7203


IndexError: list index out of range

In [16]:
for subject in all_subjects:
    model = tf.keras.models.load_model(f"/n/scratch2/ms994/cnnlstm3{subject}.h5")
    train_indices, valid_indices, test_indices = tf_mapper.get_splits_per_subject(subject)
    test_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=list(test_indices), batch_size=1, map_example=tf_mapper.map_example_to_simple_no_augment, max_queue_size=20, cache=False)
    results.append(model.evaluate(test_data, steps=len(test_indices)*40))

1840/1840 [==============================] - 61s 33ms/step - loss: 3.2096 - on_off_loss: 1.9990 - dyskinesia_loss: 0.5030 - tremor_loss: 0.7076


IndexError: list index out of range

In [27]:
len(results)

7

In [28]:
results

[[9.762943852424621, 4.463075, 3.3662014, 1.933668],
 [2.9392940325737, 1.7138741, 0.9394491, 0.28597108],
 [4.143295540571213, 2.159232, 1.0370636, 0.9469987],
 [5.611167791843414, 4.023594, 0.65771514, 0.9298563],
 [3.0847629523277282, 1.326231, 0.5457418, 1.2127903],
 [4.972854227542877, 3.2065067, 0.72244346, 1.0439029],
 [3.224292083978653, 1.9925064, 0.5111084, 0.72067696]]